In [2]:
import numpy as np
import pandas as pd
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#from mpl_toolkits import mplot3d 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from string import punctuation
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.svm import SVC
from scipy.sparse import csr_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from tensorflow import keras
from collections import Counter
from nltk.corpus import stopwords
from sklearn.svm import LinearSVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense, Activation, MaxPool1D
from tensorflow.keras.optimizers import Adam

from sklearn import set_config
set_config(display='diagram')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/justinszaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [44]:
def split_labels(data, label_feature):
    """
    Split the given column of of the data, returning the full data set (without that
    feature) and the split off feature.
    """
    return data.drop(columns=label_feature), data[label_feature]

# Frame the Problem and Look at the Big Picture 

1. Define the objective in business terms. 

    The business objective is to create a machine learning model that takes a movie goer's review and determines if it is positive or negative. The client can use this to quickly determine the overall reaction to a movie showing. 

2. How will your solution be used? 

    The solution will be used by the client to quickly determine reactions to movie showings. 

3. What are the current solutions/workarounds (if any)? 

    The current solution is to manually poll viewers to determine how they felt about the movie. This is a slow process and the review can be very mixed. 

4. How should you frame this problem (supervised/unsupervised, online/offline, ...)?

    This problem is a supervised offline sentiment analysis problem. 

5. How should performance be measured? Is the performance measure aligned with the business objective? 

    Our metric for this problem will be accuracy. The most important function of this model is to accurately measure the sentiment of the review, so it is important that our model has high accuracy.

6. What would be the minimum performance needed to reach the business objective? 

    The minimum performance required is 80% accuracy. 

7. What are comparable problems? Can you reuse experience or tools? 

    A comparable problem was using by a youtuber named Micheal Reeves. He used sentiment analysis to determine if reddit posts on r/wallstreetbets were positive or negative. We can reuse the experience but not the tools he used. There are resuable tools in the textbook for this course that we will be using. 

8. Is human expertise available? 

    No human expertise is availible at this time. 

9. How would you solve the problem manually? 

    To solve this problem manually, we would interview the movie goers individually to determine their feelings about the movie. 

10.  List the assumptions you (or others) have made so far. Verify assumptions if possible. 

    - A review can either be positive or negative. There are no "meh" reviews. 

# Get the Data 

1. List the data you need and how much you need 

    We need movie reviews and a label that determines their sentiment (1 being positive, 0 being negative). We need over 1000.

2. Find and document where you can get that data 

    The data can be gathered form the open source website Kaggle.

3. Get access authorizations 

    The webiste and data is open source, so we have full access to the data.

4. Create a workspace (with enough storage space) 

    This notebook.

5. Get the data 

    The data was downloaded from kaggle and inserted into this repository.
    Link: https://www.kaggle.com/datasets/yasserh/imdb-movie-ratings-sentiment-analysis?resource=download

6. Convert the data to a format you can easily manipulate (without changing the data itself) 

    Made into a pandas dataframe.

7. Ensure sensitive information is deleted or protected (e.g. anonymized) 

    Not applicable to this situation.

8. Check the size and type of data (time series, geographical, ...) 

    - text: object: 40000 entries
    - label: int: 400000 entries
    
9. Sample a test set, put it aside, and never look at it (no data snooping!) 

In [4]:
data = pd.read_csv("movie.csv")

In [5]:
data

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


In [53]:
train_set, test_set = train_test_split(data, stratify=data['label'], test_size=0.2, random_state=420)

# Explore the Data 

1. Copy the data for exploration, downsampling to a manageable size if necessary. 


2. Study each attribute and its characteristics: Name; Type (categorical, numerical, bounded, text, structured, ...); % of missing values; Noisiness and type of noise (stochastic, outliers, rounding errors, ...); Usefulness for the task; Type of distribution (Gaussian, uniform logarithmic, ...) 

    - text is a object datatype with no missing data.
    - label is an interger value of either 1 or a 0. 1 is positive, 0 is negative. There is no missing data. In the data set, there is almost an equal number of positive and negative labels.


3. For supervised learning tasks, identify the target attribute(s) 

    The target attribute is the label feature. It is the sentiment of the text.

4. Visualize the data 


5. Study the correlations between attributes 


6. Study how you would solve the problem manually 


7. Identify the promising transformations you may want to apply 


8. Identify extra data that would be useful (go back to “Get the Data”) 


9. Document what you have learned 
    - With statification, there is almost an equal number of text with positive/negative sentiment.

In [8]:
train_copy = train_set.copy()

In [9]:
train_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32000 entries, 33632 to 13973
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    32000 non-null  object
 1   label   32000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 750.0+ KB


In [10]:
train_copy['label'].value_counts()

0    16015
1    15985
Name: label, dtype: int64

# Bag of Words

In [61]:
all_stop_words = set(stopwords.words('english'))
all_stop_words.remove('not')
ps = PorterStemmer()


In [62]:
def BagOfWords(train_copy):
    corpus = []
    for text in train_copy['text']:
        review = text.lower()
        review = re.sub("<.+>|[^a-zA-Z]|z.+z", " ", review)
        review = review.split()
        review = [word for word in review if word not in all_stop_words]
        review = " ".join(ps.stem(word) for word in review)
        corpus.append(review)
    return corpus
    


In [63]:
corpus = BagOfWords(train_copy)
corpus

['best bit film alan pull knicker ran cut throat ra watch girl find pull knicker',
 'rain shine outsid enter movi hous make happi not come right light go settl bar ice cream move pictur begin flicker screen feel content dark back begin time sit around campfir look modern version flicker flame time per second share joy discov unknown turn twist scenario rest clan spectat not happi not write comment long live romant comedi',
 'memor line short live show view episod line introduc fratern intramur flag footbal team start use line break huddl offens instead readi break quarter back said footbal rest squad respond bet fun way break huddl oppon scratch head watch show use line ad uniqu element season never forget best season time colleg year small way due fun use line show pretti much stinker live memori pi kappa phi intramur flag footbal squad west virginia tech',
 'dawn realli surpris saw far less gruesom horror adventur expect instead director jeff lieberman conjur wonder evoc disquiet atm

In [12]:
pop_words = {}
for line in corpus:
    words = line.split()
    for word in words:
        if pop_words.get(word, -1) == -1:
            pop_words[word] = 1
        else:
            pop_words[word] += 1
pop_words

{'best': 5038,
 'bit': 2264,
 'film': 37438,
 'alan': 175,
 'pull': 629,
 'knicker': 11,
 'ran': 170,
 'cut': 897,
 'throat': 96,
 'ra': 33,
 'watch': 12653,
 'girl': 2649,
 'find': 3780,
 'rain': 176,
 'shine': 266,
 'outsid': 356,
 'enter': 278,
 'movi': 44906,
 'hous': 1470,
 'make': 10646,
 'happi': 774,
 'not': 22288,
 'come': 4545,
 'right': 2400,
 'light': 1063,
 'go': 6649,
 'settl': 93,
 'bar': 283,
 'ice': 226,
 'cream': 72,
 'move': 1668,
 'pictur': 1418,
 'begin': 1838,
 'flicker': 20,
 'screen': 1965,
 'feel': 3619,
 'content': 263,
 'dark': 1173,
 'back': 3451,
 'time': 12103,
 'sit': 941,
 'around': 2256,
 'campfir': 17,
 'look': 7001,
 'modern': 634,
 'version': 1715,
 'flame': 67,
 'per': 104,
 'second': 1476,
 'share': 347,
 'joy': 244,
 'discov': 546,
 'unknown': 205,
 'turn': 2476,
 'twist': 876,
 'scenario': 172,
 'rest': 1195,
 'clan': 28,
 'spectat': 42,
 'write': 1564,
 'comment': 1402,
 'long': 2533,
 'live': 3038,
 'romant': 637,
 'comedi': 3034,
 'memor': 484

In [13]:
keys_greater_than = []
for key in pop_words.keys():
    if pop_words.get(key) >= 1000:
        keys_greater_than.append([pop_words.get(key), key])
    
for i in sorted(keys_greater_than, reverse=True):
    print(i)

[44906, 'movi']
[37438, 'film']
[22288, 'not']
[20872, 'one']
[16604, 'like']
[12653, 'watch']
[12103, 'time']
[12071, 'good']
[11416, 'see']
[10646, 'make']
[9604, 'get']
[9149, 'would']
[9014, 'stori']
[8986, 'even']
[8766, 'realli']
[8552, 'charact']
[7854, 'well']
[7624, 'great']
[7469, 'show']
[7382, 'bad']
[7205, 'love']
[7001, 'look']
[6975, 'much']
[6714, 'peopl']
[6649, 'go']
[6632, 'think']
[6596, 'end']
[6477, 'first']
[6389, 'act']
[6274, 'made']
[6263, 'scene']
[5964, 'way']
[5776, 'seen']
[5685, 'thing']
[5656, 'could']
[5645, 'also']
[5611, 'say']
[5532, 'play']
[5294, 'know']
[5134, 'ever']
[5118, 'year']
[5038, 'best']
[4991, 'plot']
[4969, 'actor']
[4940, 'want']
[4922, 'mani']
[4676, 'better']
[4651, 'work']
[4605, 'seem']
[4587, 'never']
[4545, 'come']
[4496, 'life']
[4483, 'littl']
[4458, 'two']
[4349, 'tri']
[4320, 'take']
[4309, 'give']
[4164, 'still']
[3795, 'enjoy']
[3780, 'find']
[3753, 'man']
[3723, 'someth']
[3723, 'lot']
[3690, 'director']
[3619, 'feel']
[3

# TOP WORDS:
    [44906, 'movi']
    [37438, 'film']
    [22288, 'not']
    [20872, 'one']
    [16604, 'like']
    [12653, 'watch']
    [12103, 'time']
    [12071, 'good']
    [11416, 'see']
    [10646, 'make']
    [9604, 'get']
    [9149, 'would']
    [9014, 'stori']
    [8986, 'even']
    [8766, 'realli']
    [8552, 'charact']
    [7854, 'well']
    [7624, 'great']
    [7469, 'show']
    [7382, 'bad']
    [7205, 'love']
    [7001, 'look']
    [6975, 'much']
    [6714, 'peopl']
    [6649, 'go']

In [14]:
CountVec = CountVectorizer()
x = CountVec.fit_transform(corpus).toarray()

In [15]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [16]:
occurances = CountVec.vocabulary_
occurances

{'best': 3535,
 'bit': 3825,
 'film': 13353,
 'alan': 743,
 'pull': 30377,
 'knicker': 20827,
 'ran': 30908,
 'cut': 8777,
 'throat': 38322,
 'ra': 30704,
 'watch': 41737,
 'girl': 15210,
 'find': 13395,
 'rain': 30817,
 'shine': 34322,
 'outsid': 27695,
 'enter': 12026,
 'movi': 25527,
 'hous': 17874,
 'make': 23139,
 'happi': 16569,
 'not': 26796,
 'come': 7483,
 'right': 31998,
 'light': 22036,
 'go': 15372,
 'settl': 33965,
 'bar': 2789,
 'ice': 18220,
 'cream': 8374,
 'move': 25522,
 'pictur': 29019,
 'begin': 3248,
 'flicker': 13641,
 'screen': 33529,
 'feel': 13109,
 'content': 7865,
 'dark': 9044,
 'back': 2509,
 'time': 38465,
 'sit': 34808,
 'around': 1884,
 'campfir': 5573,
 'look': 22436,
 'modern': 25059,
 'version': 41077,
 'flame': 13554,
 'per': 28633,
 'second': 33667,
 'share': 34138,
 'joy': 19965,
 'discov': 10206,
 'unknown': 40258,
 'turn': 39456,
 'twist': 39545,
 'scenario': 33260,
 'rest': 31730,
 'clan': 7001,
 'spectat': 35776,
 'write': 42720,
 'comment': 75

In [17]:
keys_greater_than = []
for key in occurances.keys():
    if occurances.get(key) >= 10000:
        keys_greater_than.append((occurances.get(key), key))


In [18]:
for i in sorted(keys_greater_than):
    print(i)

(10000, 'diger')
(10001, 'digest')
(10002, 'digg')
(10003, 'digger')
(10004, 'diggler')
(10005, 'diggstown')
(10006, 'dighton')
(10007, 'digi')
(10008, 'digicorp')
(10009, 'digimon')
(10010, 'digisoft')
(10011, 'digit')
(10012, 'digitech')
(10013, 'digitis')
(10014, 'digiulio')
(10015, 'digiview')
(10016, 'dignifi')
(10017, 'digniti')
(10018, 'digress')
(10019, 'digust')
(10020, 'dike')
(10021, 'dikker')
(10022, 'diklaja')
(10023, 'dil')
(10024, 'dilapid')
(10025, 'dilbert')
(10026, 'dildo')
(10027, 'dilemma')
(10028, 'dilettant')
(10029, 'dilettantish')
(10030, 'dilig')
(10031, 'dilip')
(10032, 'dillan')
(10033, 'dillemma')
(10034, 'diller')
(10035, 'dilli')
(10036, 'dilling')
(10037, 'dillman')
(10038, 'dillon')
(10039, 'dillus')
(10040, 'dillut')
(10041, 'dilophosauru')
(10042, 'dilut')
(10043, 'dim')
(10044, 'dima')
(10045, 'dimaggio')
(10046, 'dime')
(10047, 'dimeco')
(10048, 'dimens')
(10049, 'dimension')
(10050, 'dimensionless')
(10051, 'dimestor')
(10052, 'diminish')
(10053, 'd

In [19]:
#plt.plot(keys_greater_than, values)
print(keys_greater_than[0:10])


[(13353, 'film'), (30377, 'pull'), (20827, 'knicker'), (30908, 'ran'), (38322, 'throat'), (30704, 'ra'), (41737, 'watch'), (15210, 'girl'), (13395, 'find'), (30817, 'rain')]


In [20]:
print(x.shape)
print(type(x))

(32000, 43320)
<class 'numpy.ndarray'>


# Prepare the Data 
1. Data cleaning: Fix/remove outliers (optional); Fill in missing values (with 0, mean, 
median...) or drop rows/columns 
2. Feature selection (optional): Drop attributes that provide no useful information 
for the task 
3. Feature engineering, where appropriate: Discretize continuous features; Decompose features (categorical, date/time, ...), 
Add promising transformations of features (log(𝑥𝑥), √𝑥𝑥, 𝑥𝑥2, ...); Aggregate features into promising new features 
4. Feature scaling: standardize or normalize features 

In [47]:
X, y = split_labels(train_set, 'label',)

In [14]:
# Creating a class that lowercase the sentences, removing <br> cases, and removing instances like zzzzzz
all_stop_words = stopwords.words('english')
all_stop_words.remove('not')
ps = PorterStemmer()

def text_editer(text):
    review = text.lower()
    review = re.sub("<.+>|[^a-zA-Z]|z.+z", " ", review)
    review = review.split()
    review = [word for word in review if word not in set(all_stop_words)]
    review = " ".join(ps.stem(word) for word in review)
    return review

class EditingText(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = pd.DataFrame(X)
        data = np.array([text_editer(text) for text in X['text']]).flatten()
        #data = data.to_numpy().reshape(-1,1)
        return data

In [15]:
preprocessor = Pipeline(steps=[
    ('EditingText', EditingText()),
    ('TfidfVectorizer', TfidfVectorizer()),
    ('TfidfVectorizerTransformer', TfidfTransformer()),
    ('StandardScaler', StandardScaler(with_mean=False)),
])

In [ ]:
preprocessor.fit(X,y)

Pipeline(steps=[('EditingText', EditingText()),
                ('TfidfVectorizer', TfidfVectorizer()),
                ('TfidfVectorizerTransformer', TfidfTransformer()),
                ('StandardScaler', StandardScaler(with_mean=False))])

In [ ]:
transformed = preprocessor.transform(X)

In [ ]:
transformed

<32000x43320 sparse matrix of type '<class 'numpy.float64'>'
	with 1745980 stored elements in Compressed Sparse Row format>

# Short-List Promising Models 

In [52]:
X, y = split_labels(train_set, 'label')

AttributeError: 'function' object has no attribute 'drop'

In [ ]:
y

33632    1
11432    1
16188    0
31733    1
29102    0
        ..
37424    1
24370    0
22580    0
29688    0
13973    0
Name: label, Length: 32000, dtype: int64

In [ ]:
MultiNB = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('MultiNB', MultinomialNB())
])
MultiNB.fit(X,y)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('EditingText', EditingText()),
                                 ('TfidfVectorizer', TfidfVectorizer()),
                                 ('TfidfVectorizerTransformer',
                                  TfidfTransformer()),
                                 ('StandardScaler',
                                  StandardScaler(with_mean=False))])),
                ('MultiNB', MultinomialNB())])

In [ ]:
y_pred = cross_val_predict(MultiNB, X, y)
accuracy_score(y,y_pred)

0.7206875

In [ ]:
BernNB = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('BernouliNB', BernoulliNB())
])
BernNB.fit(X,y)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('EditingText', EditingText()),
                                 ('TfidfVectorizer', TfidfVectorizer()),
                                 ('TfidfVectorizerTransformer',
                                  TfidfTransformer()),
                                 ('StandardScaler',
                                  StandardScaler(with_mean=False))])),
                ('BernouliNB', BernoulliNB())])

In [ ]:
y_pred = cross_val_predict(BernNB, X, y)
accuracy_score(y,y_pred)    

0.8366875

In [ ]:
scores = cross_val_score(BernNB, X, y, scoring='accuracy')
scores

array([0.840625  , 0.8353125 , 0.83609375, 0.8334375 , 0.83796875])

In [ ]:
SVC = Pipeline([
    ('preprocessor', preprocessor),
    ('SVM', LinearSVC(loss="hinge", C=1.0)),
])
SVC.fit(X, y)
y_pred = SVC.predict(X)
accuracy_score(y, y_pred)

#Severe Overfitting

/Users/justinszaro/Courses/MachineLearning/machine-learning-sentiment-analysis/.venv/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.99946875

In [55]:
X, y = split_labels(train_set, 'label')
X = X.to_numpy()
y = y.to_numpy()
X_test, X_train = X[:5000], X[5000:]
y_test, y_train = y[:5000], y[5000:]
train_set_np = tf. 
print(X_test.shape, X_train.shape)
print(y_test.shape, y_train.shape)
print(type(X_test))

(5000, 1) (27000, 1)
(5000,) (27000,)
<class 'numpy.ndarray'>


In [64]:
VOCAB_SIZE = 1000
encoder = keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_set_np.map(lambda text, label: text))

model = keras.Sequential()
model.add(encoder)
model.add(keras.layers.Flatten(input_shape=X_train[0].shape, name="input"))
model.add(keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64,  return_sequences=True)))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(2, activation=keras.activations.softmax, name="output"))

model.build(X_test.shape)
model.summary()



AttributeError: 'numpy.ndarray' object has no attribute 'map'

In [35]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [36]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10


2022-05-01 10:46:07.195625: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at lookup_table_op.cc:929 : FAILED_PRECONDITION: Table not initialized.


FailedPreconditionError: Graph execution error:

Detected at node 'sequential_7/text_vectorization_12/string_lookup_12/None_Lookup/LookupTableFindV2' defined at (most recent call last):
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/justinszaro/Courses/MachineLearning/machine-learning-sentiment-analysis/.venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/justinszaro/Courses/MachineLearning/machine-learning-sentiment-analysis/.venv/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Users/justinszaro/Courses/MachineLearning/machine-learning-sentiment-analysis/.venv/lib/python3